In [ ]:
# Chat Models: GPT-3.5 Turbo and GPT-4

In [ ]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

llm = ChatOpenAI()
output = llm.invoke('Explain quantum mechanics in one sentence.')
print(output.content)
# help(ChatOpenAI)

In [ ]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

message = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(message)
print(output.content)


### In-Memory Cache - Caching LLM Responses

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')


In [ ]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

In [ ]:
%%time
llm.invoke(prompt)


### SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.sqlite.db"))

prompt2 = "Tell me a joke"


In [ ]:
%%time
# first request (not in cache, takes longer)

llm.invoke(prompt2)



In [ ]:
%%time
# second request (cached, faster)
llm.invoke(prompt2)

## LLM Streaming


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

# 20240301 - PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in "{language}".'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='english')

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)


# 20240301 - ChatPromptTemplate

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population. Show the population in millions.')
    ]
)
messages = chat_template.format_messages(n='20', area='Africa')
print(messages)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)



20240301 - Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in "{language}".'''

prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'English'})
print(output)

In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points.'
prompt_template = PromptTemplate.from_template(template=template)
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
print(output)


# 20240301 - Sequential Chains
With Sequential chains, you can make a series of calls to one or more LLMs. You can take the output from one chain and use it as the input to another chain.

There are two types of sequential chains:
1. SimpleSequentialChain
2. General form of sequential chains

SimpleSequentialChain represents a series of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next.



In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
     template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output = overall_chain.invoke('linear regression')
print(output['output'])


# 20240301 - Agents - LangChain Agents in Action: Python REPL


In [ ]:
pip install -q langchain_experimental


In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points.')


In [ ]:
import math

fact12 = math.factorial(12)
sqrtFact12 = math.sqrt(fact12)
roundSqrtFact12 = round(sqrtFact12, 4)

print(fact12, sqrtFact12, roundSqrtFact12)

In [ ]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')

In [ ]:
response


# 20240301 - Langchain tools- DuckDuckGo and Wikipedia, Google Search
Langchain tools are like specialized apps for yout LLM. They are tiny code modules that allow it to access information and services.

These tools connect your LLM to search engines, databases, APIs, and more, expanding its knowledge and capabilities.


In [ ]:
pip install -q duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

output = search.invoke('Where was Adolf Hitler born?')
print(output)

      

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(region='en-GB', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Newport')
print(output)

In [ ]:
search2 = DuckDuckGoSearchResults()
output2 = search2.run('Freddie Mercury and Queen')
print(output2)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)
    

# Wikipedia tool

In [ ]:
pip install -q wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})


In [ ]:
wiki.invoke('Great Wall of China')

# 20240301 - Reasoning and Acting (ReAct)
ReAct is a new approach that combines reasoning (chain of thoughts prompting) and acting capabilities of LLMs.

With ReAct LLMs generate reasoning traces and task-specific actions in an interleaved manner.


# Creating a ReAct Agent

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Anser the following questions as best as you can. Translate to FRENCH.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)
# 1. PythonREPL tool
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searchin Wikipedia)
wiki_api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
ddg_search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=ddg_search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [ ]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current Prime Minister of U.K.?'
question = 'Tell me about Makaveli early life'

output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})
print(output['output'])